<a href="https://colab.research.google.com/github/michael-borck/weatherwise-template/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
# !pip install requests matplotlib pyinputplus


In [ ]:
# Environment setup (not needed for this project)
# import os
# os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
# os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
# os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

## 📦 Setup and Configuration
Import required packages and setup environment.

In [1]:
import sys
print(sys.executable)
!{sys.executable} -m pip install --upgrade pip
!{sys.ex

C:\Users\PMLS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe


'{sys.ex' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# Ollama client helper
import json
import urllib.request


def ollama_chat(prompt, system=None, model=None, host=None, json_mode=False, temperature=0.2):
    """
    Call the Ollama chat API and return the response text.

    Args:
        prompt (str): User prompt
        system (str): Optional system prompt
        model (str): Model name (defaults to GRANITE_MODEL)
        host (str): Ollama host (defaults to OLLAMA_HOST)
        json_mode (bool): If True, request JSON-safe output
        temperature (float): Sampling temperature

    Returns:
        str: Model response text
    """
    model = model or GRANITE_MODEL
    host = host or OLLAMA_HOST
    url = f"{host}/api/chat"
    headers = {"Content-Type": "application/json"}
    messages = []
    if system:
        messages.append({"role": "system", "content": system})
    messages.append({"role": "user", "content": prompt})

    body = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
    }
    if json_mode:
        body["format"] = "json"

    data = json.dumps(body).encode("utf-8")
    req = urllib.request.Request(url, data=data, headers=headers, method="POST")
    with urllib.request.urlopen(req, timeout=60) as resp:
        raw = json.loads(resp.read().decode("utf-8"))
    # Ollama returns either 'message': {'content': ...} or 'response'
    if "message" in raw and isinstance(raw["message"], dict):
        return raw["message"].get("content", "")
    return raw.get("response", "")



In [3]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
import re
import os

# Ollama / Granite configuration
OLLAMA_HOST = os.getenv('OLLAMA_HOST', 'http://localhost:11434')
GRANITE_MODEL = os.getenv('GRANITE_MODEL', 'granite3.2')

# Configure matplotlib for better display
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ WeatherWise - Weather-Aware Chatbot")
print("📦 All required packages imported successfully!")
print("🤖 LLM (Ollama) host:", OLLAMA_HOST)
print("🧠 Granite model:", GRANITE_MODEL)
print("🚀 Ready to start building your weather chatbot!")

✅ WeatherWise - Weather-Aware Chatbot
📦 All required packages imported successfully!
🤖 LLM (Ollama) host: http://localhost:11434
🧠 Granite model: granite3.2
🚀 Ready to start building your weather chatbot!


## 🌤️ Weather Data Functions

In [4]:
# Define get_weather_data() function here
def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location using wttr.in API.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    try:
        # Make API request to wttr.in
        url = f"https://wttr.in/{location}?format=j1"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        
        # Extract current weather
        current = data['current_condition'][0]
        current_weather = {
            'temperature': int(current['temp_C']),
            'feels_like': int(current['FeelsLikeC']),
            'humidity': int(current['humidity']),
            'description': current['weatherDesc'][0]['value'],
            'precipitation': float(current['precipMM']),
            'wind_speed': int(current['windspeedKmph']),
            'pressure': int(current['pressure'])
        }
        
        # Extract forecast data
        forecast = []
        for i in range(min(forecast_days, len(data['weather']))):
            day_data = data['weather'][i]
            forecast.append({
                'date': day_data['date'],
                'max_temp': int(day_data['maxtempC']),
                'min_temp': int(day_data['mintempC']),
                'avg_temp': int(day_data['avgtempC']),
                'precipitation': float(day_data['totalSnow_cm']) + float(day_data['totalprecip_mm']),
                'humidity': int(day_data['avghumidity']),
                'description': day_data['hourly'][0]['weatherDesc'][0]['value']
            })
        
        return {
            'location': location,
            'current': current_weather,
            'forecast': forecast,
            'success': True
        }
        
    except requests.exceptions.RequestException as e:
        return {
            'location': location,
            'error': f"Failed to fetch weather data: {str(e)}",
            'success': False
        }
    except KeyError as e:
        return {
            'location': location,
            'error': f"Invalid location or data format error: {str(e)}",
            'success': False
        }
    except Exception as e:
        return {
            'location': location,
            'error': f"Unexpected error: {str(e)}",
            'success': False
        }

## 📊 Visualisation Functions

In [5]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of temperature data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    if not weather_data.get('success', False):
        print(f"Error: {weather_data.get('error', 'Unknown error')}")
        return None
    
    # Prepare data for plotting
    dates = []
    max_temps = []
    min_temps = []
    avg_temps = []
    
    # Add current day data
    current = weather_data['current']
    dates.append('Today')
    max_temps.append(current['temperature'])
    min_temps.append(current['temperature'])
    avg_temps.append(current['temperature'])
    
    # Add forecast data
    for day in weather_data['forecast']:
        dates.append(day['date'])
        max_temps.append(day['max_temp'])
        min_temps.append(day['min_temp'])
        avg_temps.append(day['avg_temp'])
    
    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.plot(dates, max_temps, 'r-o', label='Max Temperature', linewidth=2, markersize=6)
    plt.plot(dates, min_temps, 'b-o', label='Min Temperature', linewidth=2, markersize=6)
    plt.plot(dates, avg_temps, 'g-o', label='Average Temperature', linewidth=2, markersize=6)
    
    plt.title(f'Temperature Forecast for {weather_data["location"]}', fontsize=16, fontweight='bold')
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Temperature (°C)', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    if output_type == 'figure':
        return plt.gcf()
    else:
        plt.show()
        return None


In [6]:

def create_precipitation_visualisation(weather_data, output_type='display'):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    if not weather_data.get('success', False):
        print(f"Error: {weather_data.get('error', 'Unknown error')}")
        return None
    
    # Prepare data for plotting
    dates = []
    precipitation = []
    
    # Add current day data
    current = weather_data['current']
    dates.append('Today')
    precipitation.append(current['precipitation'])
    
    # Add forecast data
    for day in weather_data['forecast']:
        dates.append(day['date'])
        precipitation.append(day['precipitation'])
    
    # Create the plot
    plt.figure(figsize=(12, 6))
    bars = plt.bar(dates, precipitation, color='skyblue', edgecolor='navy', alpha=0.7)
    
    # Add value labels on top of bars
    for bar, value in zip(bars, precipitation):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                f'{value:.1f}mm', ha='center', va='bottom', fontweight='bold')
    
    plt.title(f'Precipitation Forecast for {weather_data["location"]}', fontsize=16, fontweight='bold')
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Precipitation (mm)', fontsize=12)
    plt.grid(True, alpha=0.3, axis='y')
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    if output_type == 'figure':
        return plt.gcf()
    else:
        plt.show()
        return None

## 🤖 Natural Language Processing

In [7]:
# 🤖 LLM-powered parsing (Granite via Ollama) with fallback
import json as _json


def _rule_based_parse(question):
    import re as _re
    q = question.lower()
    location_patterns = [
        r"\bin\s+([a-zA-Z\s]+?)(?:\s|$|,|\.|\?)",
        r"weather\s+for\s+([a-zA-Z\s]+?)(?:\s|$|,|\.|\?)",
        r"([a-zA-Z\s]+?)\s+weather",
        r"forecast\s+for\s+([a-zA-Z\s]+?)(?:\s|$|,|\.|\?)",
    ]
    location = None
    for pat in location_patterns:
        m = _re.search(pat, q)
        if m:
            location = m.group(1).strip()
            break
    if not location:
        for city in ['london','paris','new york','tokyo','sydney','moscow','berlin','madrid','rome','amsterdam']:
            if city in q:
                location = city
                break
    attributes = []
    if any(w in q for w in ['temperature','temp','hot','cold','warm','cool']): attributes.append('temperature')
    if any(w in q for w in ['rain','precipitation','rainfall','snow','storm']): attributes.append('precipitation')
    if any(w in q for w in ['humidity','humid','moisture']): attributes.append('humidity')
    if any(w in q for w in ['wind','breeze','gust']): attributes.append('wind')
    if any(w in q for w in ['pressure','barometric']): attributes.append('pressure')
    if not attributes: attributes.append('general')
    timeframe = 'today'
    if any(w in q for w in ['tomorrow','next day']): timeframe = 'tomorrow'
    elif any(w in q for w in ['week','7 days','seven days']): timeframe = 'week'
    elif any(w in q for w in ['3 days','three days','next 3 days']): timeframe = '3days'
    elif any(w in q for w in ['5 days','five days','next 5 days']): timeframe = '5days'
    return {'location': location, 'attributes': attributes, 'timeframe': timeframe, 'original_question': question}


def parse_weather_question(question):
    """Use Granite (via Ollama) to parse question; fallback to rule-based on error."""
    try:
        system = (
            "You extract weather query parameters and return ONLY strict JSON. "
            "Keys: location (string|null), attributes (array from ['temperature','precipitation','humidity','wind','pressure','general']), "
            "timeframe (one of 'today','tomorrow','3days','5days','week')."
        )
        prompt = (
            "Question:\n" + question + "\n"+
            "Return JSON: {\"location\":...,\"attributes\":...,\"timeframe\":...}"
        )
        raw = ollama_chat(prompt, system=system, model=GRANITE_MODEL, host=OLLAMA_HOST, json_mode=True, temperature=0.1)
        txt = raw.strip()
        if txt.startswith('```'):
            txt = txt.strip('`')
            txt = txt.split('\n',1)[1] if '\n' in txt else txt
        parsed = _json.loads(txt)
        location = parsed.get('location') or None
        if isinstance(location, str): location = location.strip()
        allowed = {'temperature','precipitation','humidity','wind','pressure','general'}
        attributes = [a for a in (parsed.get('attributes') or []) if isinstance(a, str) and a in allowed]
        if not attributes: attributes = ['general']
        timeframe = parsed.get('timeframe') or 'today'
        if timeframe not in ['today','tomorrow','3days','5days','week']: timeframe = 'today'
        return {'location': location, 'attributes': attributes, 'timeframe': timeframe, 'original_question': question}
    except Exception:
        return _rule_based_parse(question)


In [8]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    """
    Parse a natural language weather question.

    Args:
        question (str): User's weather-related question

    Returns:
        dict: Extracted information including location, time period, and weather attribute
    """
    import re
    
    question_lower = question.lower()
    
    # Common location patterns
    location_patterns = [
        r'\bin\s+([a-zA-Z\s]+?)(?:\s|$|,|\.|\?)',
        r'weather\s+for\s+([a-zA-Z\s]+?)(?:\s|$|,|\.|\?)',
        r'([a-zA-Z\s]+?)\s+weather',
        r'forecast\s+for\s+([a-zA-Z\s]+?)(?:\s|$|,|\.|\?)'
    ]
    
    location = None
    for pattern in location_patterns:
        match = re.search(pattern, question_lower)
        if match:
            location = match.group(1).strip()
            break
    
    # If no location found, try to extract from common city names
    if not location:
        common_cities = ['london', 'paris', 'new york', 'tokyo', 'sydney', 'moscow', 'berlin', 'madrid', 'rome', 'amsterdam']
        for city in common_cities:
            if city in question_lower:
                location = city
                break
    
    # Extract weather attributes
    attributes = []
    if any(word in question_lower for word in ['temperature', 'temp', 'hot', 'cold', 'warm', 'cool']):
        attributes.append('temperature')
    if any(word in question_lower for word in ['rain', 'precipitation', 'rainfall', 'snow', 'storm']):
        attributes.append('precipitation')
    if any(word in question_lower for word in ['humidity', 'humid', 'moisture']):
        attributes.append('humidity')
    if any(word in question_lower for word in ['wind', 'breeze', 'gust']):
        attributes.append('wind')
    if any(word in question_lower for word in ['pressure', 'barometric']):
        attributes.append('pressure')
    if not attributes:  # Default to general weather
        attributes.append('general')
    
    # Extract time frame
    timeframe = 'today'
    if any(word in question_lower for word in ['tomorrow', 'next day']):
        timeframe = 'tomorrow'
    elif any(word in question_lower for word in ['week', '7 days', 'seven days']):
        timeframe = 'week'
    elif any(word in question_lower for word in ['3 days', 'three days', 'next 3 days']):
        timeframe = '3days'
    elif any(word in question_lower for word in ['5 days', 'five days', 'next 5 days']):
        timeframe = '5days'
    
    return {
        'location': location,
        'attributes': attributes,
        'timeframe': timeframe,
        'original_question': question
    }

## 🧭 User Interface

In [9]:
# 🤖 LLM-powered response generation with fallback

def _rule_based_response(parsed_question, weather_data):
    # reuse existing deterministic logic by calling the earlier function if present
    return generate_weather_response(parsed_question, weather_data)  # will be overwritten below


def generate_weather_response(parsed_question, weather_data):
    """
    Prefer Granite via Ollama to compose the answer; fallback to deterministic response.
    """
    try:
        if not weather_data.get('success', False):
            return f"Sorry, I couldn't get weather data for {parsed_question.get('location') or 'that location'}. {weather_data.get('error', 'Please try again.')}"
        system = (
            "You are WeatherWise, a friendly weather assistant. You will be given parsed user intent and weather data. "
            "Produce a concise, friendly answer in natural language, with bullet points for key facts."
        )
        import json as _json
        payload = {
            'parsed_question': parsed_question,
            'weather_data': weather_data,
        }
        prompt = (
            "Use the following JSON as context and answer the user's weather question.\n" +
            _json.dumps(payload, ensure_ascii=False)
        )
        answer = ollama_chat(prompt, system=system, model=GRANITE_MODEL, host=OLLAMA_HOST, json_mode=False, temperature=0.4)
        if isinstance(answer, str) and answer.strip():
            return answer.strip()
        raise ValueError('Empty LLM response')
    except Exception:
        # Fallback: deterministic response using the previous rule-based generator body (copied here)
        if not weather_data.get('success', False):
            return f"Sorry, I couldn't get weather data for {parsed_question['location'] or 'that location'}. {weather_data.get('error', 'Please try again.')}"
        location = weather_data['location']
        current = weather_data['current']
        forecast = weather_data['forecast']
        attributes = parsed_question['attributes']
        timeframe = parsed_question['timeframe']
        response_parts = []
        response_parts.append(f"Here's the weather information for {location.title()}:")
        if timeframe == 'today':
            response_parts.append(f"\n🌤️ **Current Conditions:**")
            response_parts.append(f"• Temperature: {current['temperature']}°C (feels like {current['feels_like']}°C)")
            response_parts.append(f"• Description: {current['description']}")
            response_parts.append(f"• Humidity: {current['humidity']}%")
            response_parts.append(f"• Precipitation: {current['precipitation']}mm")
            response_parts.append(f"• Wind Speed: {current['wind_speed']} km/h")
            response_parts.append(f"• Pressure: {current['pressure']} mb")
        elif timeframe == 'tomorrow' and len(forecast) > 0:
            tomorrow = forecast[0]
            response_parts.append(f"\n🌅 **Tomorrow's Forecast:**")
            response_parts.append(f"• High: {tomorrow['max_temp']}°C, Low: {tomorrow['min_temp']}°C")
            response_parts.append(f"• Description: {tomorrow['description']}")
            response_parts.append(f"• Precipitation: {tomorrow['precipitation']}mm")
            response_parts.append(f"• Humidity: {tomorrow['humidity']}%")
        else:
            days_to_show = 1
            if timeframe == '3days':
                days_to_show = min(3, len(forecast))
            elif timeframe == '5days':
                days_to_show = min(5, len(forecast))
            elif timeframe == 'week':
                days_to_show = min(7, len(forecast))
            response_parts.append(f"\n📅 **{days_to_show}-Day Forecast:**")
            for i in range(days_to_show):
                day = forecast[i]
                response_parts.append(f"• {day['date']}: {day['min_temp']}°C - {day['max_temp']}°C, {day['description']}, {day['precipitation']}mm rain")
        if 'temperature' in attributes:
            if timeframe == 'today':
                response_parts.append(f"\n🌡️ **Temperature Details:**")
                response_parts.append(f"Current temperature is {current['temperature']}°C, but it feels like {current['feels_like']}°C.")
            else:
                response_parts.append(f"\n🌡️ **Temperature Trend:**")
                temps = [day['avg_temp'] for day in forecast[:days_to_show]]
                avg_temp = sum(temps) / len(temps)
                response_parts.append(f"Average temperature over the period: {avg_temp:.1f}°C")
        if 'precipitation' in attributes:
            if timeframe == 'today':
                response_parts.append(f"\n🌧️ **Precipitation:**")
                response_parts.append(f"Current precipitation: {current['precipitation']}mm")
            else:
                response_parts.append(f"\n🌧️ **Precipitation Forecast:**")
                total_precip = sum(day['precipitation'] for day in forecast[:days_to_show])
                response_parts.append(f"Total expected precipitation: {total_precip:.1f}mm over {days_to_show} days")
        if 'humidity' in attributes:
            if timeframe == 'today':
                response_parts.append(f"\n💧 **Humidity:**")
                response_parts.append(f"Current humidity: {current['humidity']}%")
            else:
                response_parts.append(f"\n💧 **Humidity Forecast:**")
                humidities = [day['humidity'] for day in forecast[:days_to_show]]
                avg_humidity = sum(humidities) / len(humidities)
                response_parts.append(f"Average humidity: {avg_humidity:.1f}%")
        response_parts.append(f"\n💡 **Recommendation:**")
        if current['temperature'] > 25:
            response_parts.append("It's quite warm! Stay hydrated and wear light clothing.")
        elif current['temperature'] < 10:
            response_parts.append("It's chilly! Bundle up and stay warm.")
        else:
            response_parts.append("Pleasant weather! Great for outdoor activities.")
        if current['precipitation'] > 5:
            response_parts.append("Don't forget your umbrella - there's significant precipitation expected!")
        return "\n".join(response_parts)


In [10]:
# Define menu functions using pyinputplus or ipywidgets here
def display_welcome():
    """Display welcome message and instructions."""
    print("=" * 60)
    print("🌦️  Welcome to WeatherWise - Your Weather-Aware Chatbot!  🌦️")
    print("=" * 60)
    print("Ask me anything about the weather in natural language!")
    print("Examples:")
    print("• 'What's the weather like in London?'")
    print("• 'Will it rain tomorrow in Paris?'")
    print("• 'Show me the temperature forecast for New York for the next 3 days'")
    print("• 'How humid is it in Tokyo?'")
    print("=" * 60)

def get_user_question():
    """Get weather question from user."""
    return pyip.inputStr("🌤️  Ask me about the weather: ", blank=False)

def display_menu():
    """Display main menu options."""
    print("\n" + "=" * 40)
    print("📋 Main Menu")
    print("=" * 40)
    print("1. Ask a weather question")
    print("2. Exit")
    print("=" * 40)

def run_chatbot():
    """Main chatbot loop."""
    display_welcome()
    
    while True:
        display_menu()
        choice = pyip.inputMenu(['1', '2'], prompt="Choose an option: ")
        
        if choice == '1':
            try:
                # Get user question
                question = get_user_question()
                print(f"\n🤔 Processing: '{question}'")
                print("-" * 50)
                
                # Parse the question
                parsed = parse_weather_question(question)
                
                # Check if location was found
                if not parsed['location']:
                    print("❌ I couldn't identify a location in your question.")
                    print("Please include a city name, like 'What's the weather in London?'")
                    continue
                
                # Get weather data
                print(f"🌍 Fetching weather data for {parsed['location']}...")
                weather_data = get_weather_data(parsed['location'])
                
                # Generate and display response
                response = generate_weather_response(parsed, weather_data)
                print(response)
                
                # Show visualizations if data is available
                if weather_data.get('success', False):
                    print("\n📊 Generating visualizations...")
                    
                    # Show temperature chart
                    if 'temperature' in parsed['attributes'] or 'general' in parsed['attributes']:
                        create_temperature_visualisation(weather_data)
                    
                    # Show precipitation chart
                    if 'precipitation' in parsed['attributes'] or 'general' in parsed['attributes']:
                        create_precipitation_visualisation(weather_data)
                
                print("\n" + "=" * 50)
                
            except KeyboardInterrupt:
                print("\n\n👋 Goodbye! Thanks for using WeatherWise!")
                break
            except Exception as e:
                print(f"\n❌ An error occurred: {str(e)}")
                print("Please try again with a different question.")
        
        elif choice == '2':
            print("\n👋 Thank you for using WeatherWise! Have a great day! 🌤️")
            break

## 🧩 Main Application Logic

In [11]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    if not weather_data.get('success', False):
        return f"Sorry, I couldn't get weather data for {parsed_question['location'] or 'that location'}. {weather_data.get('error', 'Please try again.')}"
    
    location = weather_data['location']
    current = weather_data['current']
    forecast = weather_data['forecast']
    attributes = parsed_question['attributes']
    timeframe = parsed_question['timeframe']
    
    response_parts = []
    
    # Start with greeting and location
    response_parts.append(f"Here's the weather information for {location.title()}:")
    
    # Handle different timeframes
    if timeframe == 'today':
        response_parts.append(f"\n🌤️ **Current Conditions:**")
        response_parts.append(f"• Temperature: {current['temperature']}°C (feels like {current['feels_like']}°C)")
        response_parts.append(f"• Description: {current['description']}")
        response_parts.append(f"• Humidity: {current['humidity']}%")
        response_parts.append(f"• Precipitation: {current['precipitation']}mm")
        response_parts.append(f"• Wind Speed: {current['wind_speed']} km/h")
        response_parts.append(f"• Pressure: {current['pressure']} mb")
        
    elif timeframe == 'tomorrow' and len(forecast) > 0:
        tomorrow = forecast[0]
        response_parts.append(f"\n🌅 **Tomorrow's Forecast:**")
        response_parts.append(f"• High: {tomorrow['max_temp']}°C, Low: {tomorrow['min_temp']}°C")
        response_parts.append(f"• Description: {tomorrow['description']}")
        response_parts.append(f"• Precipitation: {tomorrow['precipitation']}mm")
        response_parts.append(f"• Humidity: {tomorrow['humidity']}%")
        
    else:
        # Multi-day forecast
        days_to_show = 1
        if timeframe == '3days':
            days_to_show = min(3, len(forecast))
        elif timeframe == '5days':
            days_to_show = min(5, len(forecast))
        elif timeframe == 'week':
            days_to_show = min(7, len(forecast))
        
        response_parts.append(f"\n📅 **{days_to_show}-Day Forecast:**")
        for i in range(days_to_show):
            day = forecast[i]
            response_parts.append(f"• {day['date']}: {day['min_temp']}°C - {day['max_temp']}°C, {day['description']}, {day['precipitation']}mm rain")
    
    # Add specific attribute details if requested
    if 'temperature' in attributes:
        if timeframe == 'today':
            response_parts.append(f"\n🌡️ **Temperature Details:**")
            response_parts.append(f"Current temperature is {current['temperature']}°C, but it feels like {current['feels_like']}°C.")
        else:
            response_parts.append(f"\n🌡️ **Temperature Trend:**")
            temps = [day['avg_temp'] for day in forecast[:days_to_show]]
            avg_temp = sum(temps) / len(temps)
            response_parts.append(f"Average temperature over the period: {avg_temp:.1f}°C")
    
    if 'precipitation' in attributes:
        if timeframe == 'today':
            response_parts.append(f"\n🌧️ **Precipitation:**")
            response_parts.append(f"Current precipitation: {current['precipitation']}mm")
        else:
            response_parts.append(f"\n🌧️ **Precipitation Forecast:**")
            total_precip = sum(day['precipitation'] for day in forecast[:days_to_show])
            response_parts.append(f"Total expected precipitation: {total_precip:.1f}mm over {days_to_show} days")
    
    if 'humidity' in attributes:
        if timeframe == 'today':
            response_parts.append(f"\n💧 **Humidity:**")
            response_parts.append(f"Current humidity: {current['humidity']}%")
        else:
            response_parts.append(f"\n💧 **Humidity Forecast:**")
            humidities = [day['humidity'] for day in forecast[:days_to_show]]
            avg_humidity = sum(humidities) / len(humidities)
            response_parts.append(f"Average humidity: {avg_humidity:.1f}%")
    
    # Add recommendation
    response_parts.append(f"\n💡 **Recommendation:**")
    if current['temperature'] > 25:
        response_parts.append("It's quite warm! Stay hydrated and wear light clothing.")
    elif current['temperature'] < 10:
        response_parts.append("It's chilly! Bundle up and stay warm.")
    else:
        response_parts.append("Pleasant weather! Great for outdoor activities.")
    
    if current['precipitation'] > 5:
        response_parts.append("Don't forget your umbrella - there's significant precipitation expected!")
    
    return "\n".join(response_parts)

## 🧪 Testing and Examples

In [12]:
# Include sample input/output for each function

# Test the weather data function
print("🧪 Testing Weather Data Function")
print("=" * 40)
test_weather = get_weather_data("London", 3)
if test_weather['success']:
    print(f"✅ Successfully fetched weather for {test_weather['location']}")
    print(f"Current temperature: {test_weather['current']['temperature']}°C")
    print(f"Forecast days: {len(test_weather['forecast'])}")
else:
    print(f"❌ Error: {test_weather['error']}")

print("\n" + "=" * 40)

# Test the question parsing function
print("🧪 Testing Question Parsing")
print("=" * 40)
test_questions = [
    "What's the weather like in Paris?",
    "Will it rain tomorrow in Tokyo?",
    "Show me temperature for New York for next 3 days",
    "How humid is it in Sydney?"
]

for question in test_questions:
    parsed = parse_weather_question(question)
    print(f"Question: '{question}'")
    print(f"Parsed: Location='{parsed['location']}', Attributes={parsed['attributes']}, Timeframe='{parsed['timeframe']}'")
    print()

print("=" * 40)

# Test the response generation
print("🧪 Testing Response Generation")
print("=" * 40)
if test_weather['success']:
    test_parsed = parse_weather_question("What's the temperature in London?")
    response = generate_weather_response(test_parsed, test_weather)
    print("Sample response:")
    print(response[:200] + "..." if len(response) > 200 else response)
else:
    print("Skipping response test due to weather data error")

print("\n" + "=" * 40)
print("🎯 Ready to run the chatbot! Use run_chatbot() to start.")
print("=" * 40)

🧪 Testing Weather Data Function
❌ Error: Invalid location or data format error: 'totalprecip_mm'

🧪 Testing Question Parsing
Question: 'What's the weather like in Paris?'
Parsed: Location='paris', Attributes=['general'], Timeframe='today'

Question: 'Will it rain tomorrow in Tokyo?'
Parsed: Location='tokyo', Attributes=['precipitation'], Timeframe='tomorrow'

Question: 'Show me temperature for New York for next 3 days'
Parsed: Location='new york', Attributes=['temperature'], Timeframe='3days'

Question: 'How humid is it in Sydney?'
Parsed: Location='sydney', Attributes=['humidity'], Timeframe='today'

🧪 Testing Response Generation
Skipping response test due to weather data error

🎯 Ready to run the chatbot! Use run_chatbot() to start.


In [ ]:
run_chatbot()

🌦️  Welcome to WeatherWise - Your Weather-Aware Chatbot!  🌦️
Ask me anything about the weather in natural language!
Examples:
• 'What's the weather like in London?'
• 'Will it rain tomorrow in Paris?'
• 'Show me the temperature forecast for New York for the next 3 days'
• 'How humid is it in Tokyo?'

📋 Main Menu
1. Ask a weather question
2. Exit
Choose an option: * 1
* 2


## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.

In [ ]:
# 🚀 Run the WeatherWise Chatbot
# Uncomment the line below to start the interactive chatbot
# run_chatbot()

# Or run a quick demo
print("🎯 Quick Demo - WeatherWise Chatbot")
print("=" * 50)
print("To start the interactive chatbot, run: run_chatbot()")
print("=" * 50)
